## Entropy

In [10]:
import math

def entropy(txt, k=1, pre=""):
    freq = {}
    n = len(txt) - k + 1
    l = 0
    for i in range(n):
        if txt[i-len(pre):i] == pre:
            if txt[i:i+k] not in freq:
                freq[txt[i:i+k]] = 0
            freq[txt[i:i+k]] += 1
            l += 1

    probs = {}
    for block, count in freq.items():
        probs[block] = count / l

    entropy = 0
    for p in probs.values():
        entropy -= p * math.log2(p)

    return entropy

In [11]:
def calc_entropy(prob_dict):
    return sum([-v*math.log2(v) for _,v in prob_dict.items()])

In [12]:
def entropy2(src):
    total = sum(src.values())
    prob = {k:v/total for k,v in src.items()}
    return calc_entropy(prob)

## Del Lab1 cambiado

In [13]:
from collections import defaultdict

def source_fromtext(txt, n=1):
    freq_packs = {}
    for i in range(len(txt) - n + 1):
        packs = txt[i:i+n]
        if packs in freq_packs:
            freq_packs[packs] += 1
        else:
            freq_packs[packs] = 1
    freq_list = [(k, v) for k, v in freq_packs.items()]
    return sorted(freq_list, key=lambda x: x[0])

## Huffman + Lab2

In [29]:
import heapq
import operator

def huffman_code(txt,src, n):
    
    #src = source_fromtext(txt, n);
    
    d_nodes = {}
    for c in src:
        d_nodes[c[0]] = 0
    
    sorted_d = sorted(src, key=lambda x: x[1]) 

    while len(sorted_d) > 1:
        new_c = sorted_d[0][0] + sorted_d[1][0]
        new_f = sorted_d[0][1] + sorted_d[1][1]
        
        for i in range(0, len(sorted_d[0][0]), n):
            package = sorted_d[0][0][i:i+n]
            d_nodes[package] += 1

        for i in range(0, len(sorted_d[1][0]), n):
            package = sorted_d[1][0][i:i+n]
            d_nodes[package] += 1
        
        sorted_d[1] = (new_c,new_f)
        sorted_d.pop(0);
        sorted_d = sorted(sorted_d, key=lambda x: x[1])
        
    return d_nodes

In [15]:
def kraft_inequality(L, N):
    suma = 0
    for l in L:
        suma += N ** -l
        
    return suma <= 1

In [16]:
def canonical_code(Lista, N, Chars):
    
    if not kraft_inequality(Lista, N):
        return 'Les longituds no satisfan Kraft: no existeix cap codi'
    
    L = sorted(Lista)
    #print(L)
    codes = []
    index = 0
    l = 0
    for length in L:
        if codes == []:
            code = list(Chars[index])
            l = 1
            while l != length:
                l += 1
                code.append(Chars[0])
        else:
            index += 1
            if (index < N):
                code[-1] = Chars[index]
            else:
                code[-1] = Chars[0]
                change_index = -2
                while code[change_index] == Chars[N-1]:
                    code[change_index] = Chars[0]
                    change_index -= 1   
                code[change_index] = Chars[Chars.index(code[change_index])+1]
                index = 0
            while l != length:
                l += 1
                code.append(Chars[0])
                index = 0
        codes.append(''.join(code))
    res = []
    for long in Lista:
        element = list(filter(lambda i: (len(i) == long), codes))
        res.append(element[0])
        codes.remove(element[0])
        
    return res

## Mean length

In [17]:
def mean_length(src, code):
    
    total_weight = sum([x[1] for x in src])
    total_length = sum([len(code[i]) * w[1] for i, w in enumerate(src)])

    return total_length / total_weight

## Arithmetic encoding

In [18]:
from math import floor
len2 = len
bin2 = bin

In [19]:
def arithmetic_decode_bin(bin,k, src,len):
    suma = 0
    src2 = []
    for i in range(len2(src)):
        suma += src[i][1]
    for i in range(len2(src)):
        src2.append((src[i][0],src[i][1]/suma))
    alpha = '0' * k
    beta = '1' * k
    gamma = bin[:k]
    usados = k
    x = ''
    cumulative_probs = [0]
    aux = 0.0
    for i in range(len2(src2)):
        aux += src2[i][1]
        cumulative_probs.append(aux)
    while len2(x) != len:
        delta = int(beta,2) - int(alpha,2) + 1
        subintervals = []
        for j in range(1, len2(cumulative_probs)):
            aux = (int(alpha,2) + int(floor(delta * cumulative_probs[j-1])),
                   int(alpha,2) + int(floor(delta * cumulative_probs[j]) - 1))
            subintervals.append(aux)
        for ind, subint in enumerate(subintervals):
            if subint[0] <= int(gamma,2) <= subint[1]:
                x += src2[ind][0]
                alpha = bin2(subint[0])[2:].zfill(k)
                beta = bin2(subint[1])[2:].zfill(k)
        if len2(x) == len:
            break
        while alpha[0] == beta[0]:
            alpha = alpha[1:] + '0'
            beta = beta[1:] + '1'
            if usados == len2(bin):
                gamma = gamma[1:] + '0'
            else:
                gamma = gamma[1:] + bin[usados]
                usados += 1
        while alpha[:2] == '01' and beta[:2] == '10':
            alpha = alpha[0] + alpha[2:] + '0'
            beta = beta[0] + beta[2:] + '1'
            if usados == len2(bin):
                gamma = gamma[0] + gamma[2:] + '0'
            else:
                gamma = gamma[0] + gamma[2:] + bin[usados]
                usados += 1
    return x



In [20]:
def arithmetic_encode_bin(str,k):
    src = source_fromtext(txt);
    suma = 0
    src2 = []
    for i in range(len(src)):
        suma += src[i][1]
    for i in range(len2(src)):
        src2.append((src[i][0],src[i][1]/suma))
   
    alpha = '0' * k
    beta = '1' * k
    c = ""
    u = 0
    cumulative_probs = [0]
    aux = 0.0
    for i in range(len(src2)):
        aux += src2[i][1]
        cumulative_probs.append(aux)
    for i in range(len(str)):
        delta = int(beta,2) - int(alpha,2) + 1
        subintervals = []
        for j in range(1, len(cumulative_probs)):
            aux = (int(alpha,2) + int(floor(delta * cumulative_probs[j-1])),
                   int(alpha,2) + int(floor(delta * cumulative_probs[j]) - 1))
            subintervals.append(aux)
        ind = [src2.index(tupla) for tupla in src2 if tupla[0] == str[i]][0]
        alpha = bin(subintervals[ind][0])[2:].zfill(k)
        beta = bin(subintervals[ind][1])[2:].zfill(k)
        while alpha[0] == beta[0]:
            c += alpha[0]
            if alpha[0] == '0':
                    c += '1' * u
            else:
                    c += '0' * u
            u = 0
            alpha = alpha[1:] + '0'
            beta = beta[1:] + '1'
        while alpha[:2] == '01' and beta[:2] == '10':
            alpha = alpha[0] + alpha[2:] + '0'
            beta = beta[0] + beta[2:] + '1'
            u += 1
    return c + '1'

In [21]:
#no esta bien
def arithmetic_encode_bin(input_str, src, k):
    suma = sum(freq for _, freq in src)
    src2 = [(s, freq/suma) for s, freq in src]
   
    alpha = '0' * k
    beta = '1' * k
    c = ""
    u = 0
    cumulative_probs = [0]
    aux = 0.0
    for i in range(len(src2)):
        aux += src2[i][1]
        cumulative_probs.append(aux)
    for i in range(len(input_str)):
        delta = int(beta,2) - int(alpha,2) + 1
        subintervals = []
        for j in range(1, len(cumulative_probs)):
            aux = (int(alpha,2) + int(floor(delta * cumulative_probs[j-1])),
                   int(alpha,2) + int(floor(delta * cumulative_probs[j]) - 1))
            subintervals.append(aux)
        ind = next(i for i, (s_j, _) in enumerate(src) if input_str[i] == s_j)
        alpha = bin(subintervals[ind][0])[2:].zfill(k)
        beta = bin(subintervals[ind][1])[2:].zfill(k)
        while alpha[0] == beta[0]:
            c += alpha[0]
            if alpha[0] == '0':
                c += '1' * u
            else:
                c += '0' * u
            u = 0
            alpha = alpha[1:] + '0'
            beta = beta[1:] + '1'
        while alpha[:2] == '01' and beta[:2] == '10':
            alpha = alpha[0] + alpha[2:] + '0'
            beta = beta[0] + beta[2:] + '1'
            u += 1
    return c + '1'


In [22]:
def arithmetic_encode_bin2(str,src,k):
    suma = 0
    src2 = []
    for i in range(len2(src)):
        suma += src[i][1]
    for i in range(len2(src)):
        src2.append((src[i][0],src[i][1]/suma))
    alpha = '0' * k
    beta = '1' * k
    c = ""
    u = 0
    cumulative_probs = [0]
    aux = 0.0
    for i in range(len(src2)):
        aux += src2[i][1]
        cumulative_probs.append(aux)
    for i in range(0, len(str), 2):
        if (i < len(str) - 1):
            delta = int(beta,2) - int(alpha,2) + 1
            subintervals = []
            for j in range(1, len(cumulative_probs)):
                aux = (int(alpha,2) + int(floor(delta * cumulative_probs[j-1])),
                       int(alpha,2) + int(floor(delta * cumulative_probs[j]) - 1))
                subintervals.append(aux)
            ind = [src2.index(tupla) for tupla in src2 if tupla[0] == (str[i] + str[i + 1])][0]
            alpha = bin(subintervals[ind][0])[2:].zfill(k)
            beta = bin(subintervals[ind][1])[2:].zfill(k)
            while alpha[0] == beta[0]:
                c += alpha[0]
                if alpha[0] == '0':
                        c += '1' * u
                else:
                        c += '0' * u
                u = 0
                alpha = alpha[1:] + '0'
                beta = beta[1:] + '1'
            while alpha[:2] == '01' and beta[:2] == '10':
                alpha = alpha[0] + alpha[2:] + '0'
                beta = beta[0] + beta[2:] + '1'
                u += 1
    return c + '1'

## Lab2 Encode & Decode

In [32]:
def encode (txta, corr):
    
    txt_encoded = ''
    i, j = 0, 0
    
    while j <= len(txta):
        substring = txta[i:j]
        if substring in corr:
            txt_encoded += corr[substring]
            i = j
        j += 1

    if i != len(txta): # all the text could not be processed
        return 'Message could not be encoded'
    
    return txt_encoded

In [24]:
def decode(txta, corr):
    
    txt_decoded = ''
    i, j = 0, 1
    
    while j <= len(txta):
        substring = txta[i:j]
        if substring in corr.values():
            key = list(corr.keys())[list(corr.values()).index(substring)]
            txt_decoded += key
            i = j
        j += 1

    if i != len(txta):
        return 'Message could not be decoded'
    
    return txt_decoded

## Product probabilities

In [25]:
def product_prob(src):
    dict2 = {}
    src2 = dict(src)
    for k, v in src2.items():
        for l, m in src2.items():
            dict2[k+l] = v*m
    my_list = [(k, v) for k, v in dict2.items()]
    return my_list

## Tests

In [26]:
quijote = open("quijote_clean.txt","r",encoding="utf-8").read(); quijote[:1000]

'el ingenioso hidalgo don quijote de la mancha tasa yo juan gallo de andrada escribano de camara del rey nuestro senor de los que residen en su consejo certifico y doy fe que habiendo visto por los senores del un libro intitulado el ingenioso hidalgo de la mancha compuesto por miguel de cervantes saavedra tasaron cada pliego del dicho libro a tres maravedis y medio el cual tiene ochenta y tres pliegos que al dicho precio monta el dicho libro docientos y noventa maravedis y medio en que se ha de vender en papel y dieron licencia para que a este precio se pueda vender y mandaron que esta tasa se ponga al principio del dicho libro y no se pueda vender sin ella y para que dello conste di la presente en valladolid a veinte dias del mes de deciembre de mil y seiscientos y cuatro anos juan gallo de andrada testimonio de las erratas este libro no tiene cosa digna que no corresponda a su original en testimonio de lo haber correcto di esta fee en el colegio de la madre de dios de los teologos de

In [72]:
[entropy(quijote,k) for k in range(1,5)]

[3.9688875658964196, 7.004395616708034, 9.58395130083852, 11.723763678102449]

In [73]:
[entropy(quijote,k)/k for k in range(1,5)]

[3.9688875658964196, 3.502197808354017, 3.1946504336128396, 2.930940919525612]

In [74]:
entropy(quijote,1," ")

4.07680021049908

In [75]:
[entropy(quijote,1,"q"), entropy(quijote,1,"a"), entropy(quijote,1,"j"), entropy(quijote,1,"m")]

[0.0, 3.2018434371133657, 1.715405240640153, 2.5628583734966432]

In [76]:
[entropy(quijote,2,pre="res"), entropy(quijote,1,pre="quij"), entropy(quijote,3,pre="al"), entropy(quijote,1,pre="espadas")]

[4.329577792128523, 0.06840583225936889, 7.215960486296072, 0.0]

In [77]:
[entropy(quijote,k+1) - entropy(quijote,k) for k in range(1,5)]

[3.035508050811614, 2.5795556841304856, 2.1398123772639295, 1.8052454910228128]

In [78]:
src = source_fromtext(quijote); print(src)

[(' ', 381208), ('a', 200499), ('b', 24147), ('c', 59437), ('d', 87240), ('e', 229191), ('f', 7581), ('g', 17225), ('h', 19920), ('i', 90077), ('j', 10530), ('l', 89143), ('m', 44658), ('n', 112683), ('o', 162514), ('p', 35465), ('q', 32483), ('r', 100955), ('s', 125728), ('t', 61749), ('u', 79560), ('v', 17856), ('w', 2), ('x', 377), ('y', 25115), ('z', 6491)]


In [27]:
res = huffman_code(quijote, 1)
print(res)
huf = [(key, value) for key, value in zip(res.keys(), canonical_code(res.values(), 2, ['0', '1']))]
print(huf)

{' ': 2, 'a': 3, 'b': 7, 'c': 5, 'd': 5, 'e': 3, 'f': 9, 'g': 7, 'h': 7, 'i': 5, 'j': 8, 'l': 5, 'm': 6, 'n': 4, 'o': 4, 'p': 6, 'q': 6, 'r': 5, 's': 4, 't': 5, 'u': 5, 'v': 7, 'w': 11, 'x': 11, 'y': 7, 'z': 10}
[(' ', '00'), ('a', '010'), ('b', '1111010'), ('c', '10110'), ('d', '10111'), ('e', '011'), ('f', '111111110'), ('g', '1111011'), ('h', '1111100'), ('i', '11000'), ('j', '11111110'), ('l', '11001'), ('m', '111010'), ('n', '1000'), ('o', '1001'), ('p', '111011'), ('q', '111100'), ('r', '11010'), ('s', '1010'), ('t', '11011'), ('u', '11100'), ('v', '1111101'), ('w', '11111111110'), ('x', '11111111111'), ('y', '1111110'), ('z', '1111111110')]


In [82]:
cod = [y for x,y in huf];
entropy(quijote,1), mean_length(src,cod)

(3.9688875658964196, 4.01764734394614)

In [96]:
src2 = source_fromtext(quijote,2);
print(src2)

[(' a', 30582), (' b', 5547), (' c', 26710), (' d', 43510), (' e', 35913), (' f', 5025), (' g', 3813), (' h', 12512), (' i', 2723), (' j', 1283), (' l', 32728), (' m', 21278), (' n', 11545), (' o', 6064), (' p', 25482), (' q', 27349), (' r', 6915), (' s', 30083), (' t', 15198), (' u', 4214), (' v', 11166), (' w', 2), (' x', 82), (' y', 21251), (' z', 232), ('a ', 69563), ('aa', 10), ('ab', 10283), ('ac', 5158), ('ad', 12488), ('ae', 435), ('af', 279), ('ag', 1846), ('ah', 585), ('ai', 926), ('aj', 1069), ('al', 14904), ('am', 5728), ('an', 24135), ('ao', 72), ('ap', 1372), ('aq', 1925), ('ar', 17978), ('as', 23324), ('at', 2606), ('au', 1593), ('av', 1159), ('ax', 1), ('ay', 1410), ('az', 1650), ('b ', 7), ('ba', 7301), ('bc', 4), ('bd', 1), ('be', 3074), ('bi', 4571), ('bj', 9), ('bl', 1645), ('bo', 1451), ('br', 4281), ('bs', 18), ('bt', 1), ('bu', 1784), ('c ', 15), ('ca', 10986), ('cc', 53), ('ce', 7034), ('ch', 6368), ('ci', 10132), ('cl', 479), ('cm', 1), ('co', 16171), ('cr', 16

In [85]:
res2 = huffman_code(quijote, 2)
huf2 = [(key, value) for key, value in zip(res2.keys(), canonical_code(res2.values(), 2, ['0', '1']))]
print(huf2)

[(' a', '001000'), (' b', '10110000'), (' c', '001001'), (' d', '001010'), (' e', '001011'), (' f', '110101110'), (' g', '110101111'), (' h', '0111000'), (' i', '1110110000'), (' j', '11111001010'), (' l', '001100'), (' m', '0111001'), (' n', '0111010'), (' o', '10110001'), (' p', '001101'), (' q', '001110'), (' r', '10110010'), (' s', '001111'), (' t', '0111011'), (' u', '110110000'), (' v', '0111100'), (' w', '11111111111111101100'), (' x', '111111111101000'), (' y', '0111101'), (' z', '1111111011000'), ('a ', '00000'), ('aa', '111111111111101100'), ('ab', '10110011'), ('ac', '110110001'), ('ad', '0111110'), ('ae', '111111001110'), ('af', '1111111011001'), ('ag', '1110110001'), ('ah', '111111001111'), ('ai', '11111001011'), ('aj', '11111001100'), ('al', '0111111'), ('am', '10110100'), ('an', '010000'), ('ao', '111111111101001'), ('ap', '1110110010'), ('aq', '1110110011'), ('ar', '1000000'), ('as', '010001'), ('at', '1110110100'), ('au', '1110110101'), ('av', '11111001101'), ('ax', '1

In [88]:
cod2 = [y for x,y in huf2];
entropy(quijote,2), mean_length(src2,cod2)

(7.004395616708034, 7.031574318947213)

In [126]:
txt="001110001000000000001000"; src = source_fromtext(txt); print(src)

[('0', 19), ('1', 5)]


In [142]:
txt="Setze jutges d'un jutjat mengen fetge d'un penjat.";
src = source_fromtext(txt);
print(src);

[(' ', 7), ("'", 2), ('.', 1), ('S', 1), ('a', 2), ('d', 2), ('e', 8), ('f', 1), ('g', 3), ('j', 4), ('m', 1), ('n', 5), ('p', 1), ('s', 1), ('t', 6), ('u', 4), ('z', 1)]


In [143]:
cod = arithmetic_encode_bin(txt,src,8);
print(cod)

001101010011001110100111101100011100010001010010001101110110101100100001110011000111010111101101110001110010000011000001100110111000111000011101110001011111000000110011100110101111111111


In [144]:
txt_decoded = arithmetic_decode_bin(cod,8,src,len(txt));
print(txt_decoded)
txt_decoded == txt

Setze jutges d'un jutjat mengen fetge d'un penjat.


True

In [145]:
len(txt)*entropy(txt,1),len(cod)

(184.66702205340573, 186)

In [22]:
txt = "En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna diferencia en los autores que deste caso escriben; aunque, por conjeturas verosímiles, se deja entender que se llamaba Quejana. Pero esto importa poco a nuestro cuento; basta que en la narración dél no se salga un punto de la verdad.\nEs, pues, de saber que este sobredicho hidalgo, los ratos que estaba ocioso, que eran los más del año, se daba a leer libros de caballerías, con tanta afición y gusto, que olvidó casi de todo punto el ejercicio de la caza, y aun la administración de su hacienda. Y llegó a tanto su curiosidad y desatino en esto, que vendió muchas hanegas de tierra de sembradura para comprar libros de caballerías en que leer, y así, llevó a su casa todos cuantos pudo haber dellos; y de todos, ningunos le parecían tan bien como los que compuso el famoso Feliciano de Silva, porque la claridad de su prosa y aquellas entricadas razones suyas le parecían de perlas, y más cuando llegaba a leer aquellos requiebros y cartas de desafíos, donde en muchas partes hallaba escrito: La razón de la sinrazón que a mi razón se hace, de tal manera mi razón enflaquece, que con razón me quejo de la vuestra fermosura. Y también cuando leía: ...los altos cielos que de vuestra divinidad divinamente con las estrellas os fortifican, y os hacen merecedora del merecimiento que merece la vuestra grandeza.\nCon estas razones perdía el pobre caballero el juicio, y desvelábase por entenderlas y desentrañarles el sentido, que no se lo sacara ni las entendiera el mesmo Aristóteles, si resucitara para sólo ello. No estaba muy bien con las heridas que don Belianís daba y recebía, porque se imaginaba que, por grandes maestros que le hubiesen curado, no dejaría de tener el rostro y todo el cuerpo lleno de cicatrices y señales. Pero, con todo, alababa en su autor aquel acabar su libro con la promesa de aquella inacabable aventura, y muchas veces le vino deseo de tomar la pluma y dalle fin al pie de la letra, como allí se promete; y sin duda alguna lo hiciera, y aun saliera con ello, si otros mayores y continuos pensamientos no se lo estorbaran. Tuvo muchas veces competencia con el cura de su lugar -que era hombre docto, graduado en Sigüenza-, sobre cuál había sido mejor caballero: Palmerín de Ingalaterra o Amadís de Gaula; mas maese Nicolás, barbero del mesmo pueblo, decía que ninguno llegaba al Caballero del Febo, y que si alguno se le podía comparar, era don Galaor, hermano de Amadís de Gaula, porque tenía muy acomodada condición para todo; que no era caballero melindroso, ni tan llorón como su hermano, y que en lo de la valentía no le iba en zaga.\nEn resolución, él se enfrascó tanto en su letura, que se le pasaban las noches leyendo de claro en claro, y los días de turbio en turbio; y así, del poco dormir y del mucho leer, se le secó el celebro, de manera que vino a perder el juicio. Llenósele la fantasía de todo aquello que leía en los libros, así de encantamentos como de pendencias, batallas, desafíos, heridas, requiebros, amores, tormentas y disparates imposibles; y asentósele de tal modo en la imaginación que era verdad toda aquella máquina de aquellas sonadas soñadas invenciones que leía, que para él no había otra historia más cierta en el mundo. Decía él que el Cid Ruy Díaz había sido muy buen caballero, pero que no tenía que ver con el Caballero de la Ardiente Espada, que de sólo un revés había partido por medio dos fieros y descomunales gigantes. Mejor estaba con Bernardo del Carpio, porque en Roncesvalles había muerto a Roldán el encantado, valiéndose de la industria de Hércules, cuando ahogó a Anteo, el hijo de la Tierra, entre los brazos. Decía mucho bien del gigante Morgante, porque, con ser de aquella generación gigantea, que todos son soberbios y descomedidos, él solo era afable y bien criado. Pero, sobre todos, estaba bien con Reinaldos de Montalbán, y más cuando le veía salir de su castillo y robar cuantos topaba, y cuando en allende robó aquel ídolo de Mahoma que era todo de oro, según dice su historia. Diera él, por dar una mano de coces al traidor de Galalón, al ama que tenía, y aun a su sobrina de añadidura.\nEn efeto, rematado ya su juicio, vino a dar en el más estraño pensamiento que jamás dio loco en el mundo; y fue que le pareció convenible y necesario, así para el aumento de su honra como para el servicio de su república, hacerse caballero andante, y irse por todo el mundo con sus armas y caballo a buscar las aventuras y a ejercitarse en todo aquello que él había leído que los caballeros andantes se ejercitaban, deshaciendo todo género de agravio, y poniéndose en ocasiones y peligros donde, acabándolos, cobrase eterno nombre y fama. Imaginábase el pobre ya coronado por el valor de su brazo, por lo menos, del imperio de Trapisonda; y así, con estos tan agradables pensamientos, llevado del estraño gusto que en ellos sentía, se dio priesa a poner en efeto lo que deseaba.\nY lo primero que hizo fue limpiar unas armas que habían sido de sus bisabuelos, que, tomadas de orín y llenas de moho, luengos siglos había que estaban puestas y olvidadas en un rincón. Limpiólas y aderezólas lo mejor que pudo, pero vio que tenían una gran falta, y era que no tenían celada de encaje, sino morrión simple; mas a esto suplió su industria, porque de cartones hizo un modo de media celada, que, encajada con el morrión, hacían una apariencia de celada entera. Es verdad que para probar si era fuerte y podía estar al riesgo de una cuchillada, sacó su espada y le dio dos golpes, y con el primero y en un punto deshizo lo que había hecho en una semana; y no dejó de parecerle mal la facilidad con que la había hecho pedazos, y, por asegurarse deste peligro, la tornó a hacer de nuevo, poniéndole unas barras de hierro por de dentro, de tal manera que él quedó satisfecho de su fortaleza; y, sin querer hacer nueva experiencia della, la diputó y tuvo por celada finísima de encaje.\nFue luego a ver su rocín, y, aunque tenía más cuartos que un real y más tachas que el caballo de Gonela, que tantum pellis et ossa fuit, le pareció que ni el Bucéfalo de Alejandro ni Babieca el del Cid con él se igualaban. Cuatro días se le pasaron en imaginar qué nombre le pondría; porque, según se decía él a sí mesmo, no era razón que caballo de caballero tan famoso, y tan bueno él por sí, estuviese sin nombre conocido; y ansí, procuraba acomodársele de manera que declarase quién había sido, antes que fuese de caballero andante, y lo que era entonces; pues estaba muy puesto en razón que, mudando su señor estado, mudase él también el nombre, y le cobrase famoso y de estruendo, como convenía a la nueva orden y al nuevo ejercicio que ya profesaba. Y así, después de muchos nombres que formó, borró y quitó, añadió, deshizo y tornó a hacer en su memoria e imaginación, al fin le vino a llamar Rocinante: nombre, a su parecer, alto, sonoro y significativo de lo que había sido cuando fue rocín, antes de lo que ahora era, que era antes y primero de todos los rocines del mundo.\nPuesto nombre, y tan a su gusto, a su caballo, quiso ponérsele a sí mismo, y en este pensamiento duró otros ocho días, y al cabo se vino a llamar don Quijote; de donde -como queda dicho- tomaron ocasión los autores desta tan verdadera historia que, sin duda, se debía de llamar Quijada, y no Quesada, como otros quisieron decir. Pero, acordándose que el valeroso Amadís no sólo se había contentado con llamarse Amadís a secas, sino que añadió el nombre de su reino y patria, por Hepila famosa, y se llamó Amadís de Gaula, así quiso, como buen caballero, añadir al suyo el nombre de la suya y llamarse don Quijote de la Mancha, con que, a su parecer, declaraba muy al vivo su linaje y patria, y la honraba con tomar el sobrenombre della.\nLimpias, pues, sus armas, hecho del morrión celada, puesto nombre a su rocín y confirmándose a sí mismo, se dio a entender que no le faltaba otra cosa sino buscar una dama de quien enamorarse; porque el caballero andante sin amores era árbol sin hojas y sin fruto y cuerpo sin alma. Decíase él a sí:\n-Si yo, por malos de mis pecados, o por mi buena suerte, me encuentro por ahí con algún gigante, como de ordinario les acontece a los caballeros andantes, y le derribo de un encuentro, o le parto por mitad del cuerpo, o, finalmente, le venzo y le rindo, ¿no será bien tener a quien enviarle presentado y que entre y se hinque de rodillas ante mi dulce señora, y diga con voz humilde y rendido: ''Yo, señora, soy el gigante Caraculiambro, señor de la ínsula Malindrania, a quien venció en singular batalla el jamás como se debe alabado caballero don Quijote de la Mancha, el cual me mandó que me presentase ante vuestra merced, para que la vuestra grandeza disponga de mí a su talante''?\n¡Oh, cómo se holgó nuestro buen caballero cuando hubo hecho este discurso, y más cuando halló a quien dar nombre de su dama! Y fue, a lo que se cree, que en un lugar cerca del suyo había una moza labradora de muy buen parecer, de quien él un tiempo anduvo enamorado, aunque, según se entiende, ella jamás lo supo, ni le dio cata dello. Llamábase Aldonza Lorenzo, y a ésta le pareció ser bien darle título de señora de sus pensamientos; y, buscándole nombre que no desdijese mucho del suyo, y que tirase y se encaminase al de princesa y gran señora, vino a llamarla Dulcinea del Toboso, porque era natural del Toboso; nombre, a su parecer, músico y peregrino y significativo, como todos los demás que a él y a sus cosas había puesto."
cod = arithmetic_encode_bin(txt,23);
print(cod)

...


## Atenea

In [146]:
txt = "a couple of yards in the air and then delivered a war whoop and set off round and round the room banging against furniture upsetting flower pots and making general havoc next he rose on his hind feet and pranced around in a frenzy of enjoyment with his head over his shoulder and his voice proclaiming his unappeasable happiness then he went tearing around the house again spreading chaos and destruction in his path aunt polly entered in time to see him throw a few double summersets deliver a final mighty hurrah and sail through the open window carrying the rest of the flower pots with him the old lady stood petrified with astonishment peering"
src = [(' ', 72190), ('a', 23858), ('b', 5017), ('c', 6697), ('d', 15027), ('e', 36243), ('f', 6113), ('g', 6699), ('h', 19838), ('i', 19165), ('j', 662), ('k', 3070), ('l', 12294), ('m', 7309), ('n', 20475), ('o', 23601), ('p', 4825), ('q', 180), ('r', 15584), ('s', 18060), ('t', 29363), ('u', 9107), ('v', 2433), ('w', 8111), ('x', 412), ('y', 6809), ('z', 158)];
cod = arithmetic_encode_bin(txt,src,21);
cod

'001100100110011110100001101111101000111010111100110011111011011000011101000111111010011011111111101010101101010011001000100111110000100110000011000011010101101010111111100110110111000110111001100011001010110010101111011000111011011000111101100111001111110100000001110110011010010010011010011011010011011000011001111111001010011000110011011011011100111111111000100001101111001101100001111000011110111110010000101110001100100010110010101100110010100101011011100001110011111011010001110001011010101111111011000001100111000001011111010101100110101010011010101101001101011100001001010010000101001101011111000011000111000010111110001010001100101011110010100000110011001010011010000111010101010100011011110101111001011000111000100010011010101011011101100001111100000011100101100011011001011000011100111110011101101011000000000011111110111001011100110010111001001000100100111010110111010000010110000000101110111000101011001001000000101100101011010011011010100100101000010101010100000101011100001111001011111

In [112]:
txt = "111111110111111111111111111011111111111111111111111100110111111101111111110111111111111111111111111111111101110111111111111111111111111111111111110111111101111111011111111111011111111111111111111111101111110111111101111111111111111111111111111111111011110101111101011111011110011111111111011101111101111111111111111111111111011111111111111111111111111111101111111111011101111111111100111111111111111111111111111111111101011111111111111111110111111111111101111011111110111111111101111111111111111111111111111111110111111111111111111111111111101111011101111111111111111111101111111111111111111111111111111110101111111111111101111111111111111111101111111111111111111111110101111111100111011111111111011111011111111111111101111111111111111111111111111111011111111111110111111101111111111111111111111110111101111111001101111101111111110011111111111111111101111101111101010111111111111111101111111111111111111111101111111111101011111111111111111101111111111110101111111111111111111111111111111111111101011111111011111111111111111111111111110110111101111111110011111011011111110111101111111111111111111101111111111111111100110111101111110111111111111101111111111111111111011111111111110111111011111111111100111111111111111111111111110111011111111011001111111111011111111111111111111111001101111111011101111111011111111111111111011110111110011111011111111111111101111111011110111111111111110111111111011111011111111111111111001101111100111111111111011111101111011111111111110011110101111111011111111111111111111111111111101101111111111101111111111111111111111111111111111111111111111111111111011111111101011111110011111111101111110111111001111111111111111111111111011101111011111111111101111111111111110111111111111111110101111111111111111111111111111111111110111111110111111111111111111111110011111111111111111111111011111111111111110111011101111011111111011101111111110111110111110101111111111111011111111111011111111111011101111101111111111111111111001111011111111111111111111101111111111111111111111111111101111111011100111111111111110111111111110011111001111111011111111111111111111111111111111110111111011100111111011101111001111111111111111111111111111111111111111111111101110111111111110111111111110111111111111111111101111111100111111111111111111111111111111111011111110111111110111100111111111111111111111111110111111111111111111111110011111111111011111111111111111111111111111111111111010111111111110111111101111111111111111111111111111111111111111111111111110111111111111110111111111111111111111111111111011101111111110111011111010111111111111111110111011111111111111100111111010111000111101111111111011111111101111111111101111111111101011111111111111111111111111111110111111111111111111011111101111111111111111101110111111111110111110111111111011111111111111111110111111101110111111111111111111111111111111111111111111111111111"
src = [('00', 37), ('01', 288), ('10', 78), ('11', 1947)]
cod = arithmetic_encode_bin2(txt,src,15);
cod

'100101010101000010100110010101111100100100101010010001000011011011101100001100000100001111111111001011010011110000101001110001001101100111001001010001111110111110100001100000000011011010010011111110001110001001111110010001110011000110000111001110100001111001001010010000100010001101000111110000100010110011001011111111111110101011100110011011001010100101101011011011100000100011011011011111110010101100001111000001100011110001100010110110100100110011101000111011110101001011010100001101100100101000110011011010100001010001010001110101101111101101101010000001100110111010010011111010110010010100011000100101101011000111111010000110101011111101011101000000111001100101011111001011001100000110111000110111101011010010100111110011100100111010001010011001011010100001100101001011010110110010000101111001011101100001100000001110010100111101010000010011111110010111001010111110101101100101010010001101001100001101011011011111000101110011110001101100010110110101111100111111001001010111101111010100111100001

In [35]:
txt = "of himself the clock chimed eleven but he did not hear it and then there came mingling with his half formed dreams a most melancholy caterwauling the raising of a neighboring window disturbed him a cry of scat you devil and the crash of an empty bottle against the back of his aunts woodshed brought him wide awake and a single minute later he was dressed and out of the window and creeping along the roof of the ell on all fours he meowd with caution once or twice as he went then jumped to the roof of the woodshed and thence to"
src = [(' ', 72190), ('a', 23858), ('b', 5017), ('c', 6697), ('d', 15027), ('e', 36243), ('f', 6113), ('g', 6699), ('h', 19838), ('i', 19165), ('j', 662), ('k', 3070), ('l', 12294), ('m', 7309), ('n', 20475), ('o', 23601), ('p', 4825), ('q', 180), ('r', 15584), ('s', 18060), ('t', 29363), ('u', 9107), ('v', 2433), ('w', 8111), ('x', 412), ('y', 6809), ('z', 158)];
res = huffman_code(txt, src, 1)
huf = [(key, value) for key, value in zip(res.keys(), canonical_code(res.values(), 2, ['0', '1']))]
corr = dict(huf)
encode(txt,corr)

'100111100000011001111110101010010111001111000001011011001010011011111001100111011111111100011011101100111111010010111000000101110010101111111100101100000110110111100101100011001010011000011111000001000100110110001100101010011010000111101100010010001100000101101100101100000101101100101110100101001101110100111010010100111010011110001110011100101111000111001001111010111101101100001100111101000011001001100111100000111000100111010111010010111000001100011010010101001110101010000100001110101001101010110011101001011100101001000110111011010011100111111000110111010010110101110101111010100111100110010111100011100100101101100101001101001000111101001111000111001001001111000000100001000010101111110010110110110100111010011110001110010011110101111000110001001111101001100001111010101111110011010110110010111000000110011111101000010000110111110101111100010011110000010101101110100101100111110100111110000110000101111111100111110010001001000110000010110110010100110111110100100101001100010011110000001001000

In [149]:
txt = "0111101010000111010100000010111010010000111000111001010110101000101101101001110101111110111110001011001100001010001010100111110101000000011111000010110011011111110000111100101001101110001001001000111100010100000001100100011001011001001000011001000110111101111000011011110101101101000011111100011111001111010000011000000001000100000110111100010010001010011000101001111110010110111010111001101111111101110000101111011110010111100010001100010101001110011110000000101100001100011101000011110110101101111111111000101100111001100101100000000001100011101101000000101100111111000010100101101010001101110000110111111100011100110110000100111010110110100111100110001111011111011111101000001001010111111000010000111100100100100110010110000100000100000110100010010010011111000011110101010111011001110101010011011101011110011000001011010101010001101010100001001010011111010000011001100010000001110000100010100110111100000111011111011110100100010011111111101111010011001100000011100000110101011110100101011011110111100110111110011001100000111010111010001010110001111111111000101110011001100111000010111101111001010011001001000110011011010010011101110111001100111101011011100011011111111011110000011101111101100011111110101100010111011100000010001011100111110100001100001001100011101011001000100110001001011111101000111110010010011110100100000101111000101010101010110001001010100010111100101110110000110100100010101000011110011011011011011110001001001101111001010010110001100111101001100011001010011010111010010001111000000101101110000000111011001001011110101110110111000000110001000011010001011101001110011000101010101111000110111110111001100110010010111111011100110000000000001110110110000001001101110110101001001010011010110110101001111101001000000001001110100111101001101110001111100010100010000101110011001100110101110110011010100100110101010010011100100100010000000101110110010010001001110001110000000011111101110100111111010000000000111100101000101000001011100001000010101011101110000011111001010000111010101111001011110101000111110000001111011101001100101111110000111011010100111111110001010100110001101011101000111000010000111000011101011010100011011111011011110000001010100000111000011011010100001100100101111011011001011011110111111010101100011110000010011100001010010111110110000111111011101111010110010001001110011111111010110100100000100011011001001111110000001101101011001010101111110001111010110001110101001100001100001001100110111110110010010111100000011011111010011011000000000101111000000010001000001100010001111110100100111101001010100011011110111001101010000011000010101010101101001010000000101100100111000001100001111011000011101010101100011100000111100000001011001101100001001111010001100110100001100100001010010000011100000010100010010100010101000110100101010101100100001111000011001001100111101100111100010010111000110101111011110010111000010111110101011011010111001001010010100001001100001011"
src = [(' ', 72190), ('a', 23858), ('b', 5017), ('c', 6697), ('d', 15027), ('e', 36243), ('f', 6113), ('g', 6699), ('h', 19838), ('i', 19165), ('j', 662), ('k', 3070), ('l', 12294), ('m', 7309), ('n', 20475), ('o', 23601), ('p', 4825), ('q', 180), ('r', 15584), ('s', 18060), ('t', 29363), ('u', 9107), ('v', 2433), ('w', 8111), ('x', 412), ('y', 6809), ('z', 158)];
txt_decoded = arithmetic_decode_bin(txt,23,src,713);
print(txt_decoded)

he said well ill go back to the widder for a month and tackle it and see if i can come to stand it if youll let me blong to the gang tom all right huck its a whiz come along old chap and ill ask the widow to let up on you a little huck will you tom now will you thats good if shell let up on some of the roughest things ill smoke private and cuss private and crowd through or bust when you going to start the gang and turn robbers oh right off well get the boys together and have the initiation tonight maybe have the which have the initiation whats that its to swear to stand by one another and never tell the gangs secrets even if youre chopped all to flinders and kill anybody and all his family that hurts one


In [ ]:
txt = "111111011100000101111000000000000110101101110100100001010000000111110001010011101001001010101110101111000101000110000101011111001010001100101010100010000111011011100010111001111001111011101111111011101000100110101001010111001100110010000000100111001011110110001010000011110100111010101011011011100111011001100101100010111101000010110011010110010011100011111100011101001001000111111000011010100000111101010000101000101100011100001011111100101000100001100001111110001010110110100001010111101011011010110100111010100110001001001011001010101111100011110101000110110010101100011110001010011101100110011100010011110011000001000010110111101101111000110110001111110011101111111010010100001001110110101010001110001000000001100110011101100010010110111001110100100110111000110110011110000011000110111101010011110000001000000111101101101110011100101011001111101011010011000101111101001101010010011011011010000111111010000100001000001011000101011010011100001010100001001001000111000111001010100100001101010101001101111111111000001111011100111111100011010011100000100011101111001100100110000010011101111010101011011110000001010111010001001100111011011000110111010000100110110000011010111010010011000111110000011110011100000000000100011011011110001100001010110001101010100000100110001011101101100101111010010101000010100100001000101110010100011111101001101111111111100011000100000101110111101101001101111101011000011000011011111110111111011011100010110100001010001000000101011010010100001001000010011100110111110100000011100001000111001010111100100100101001101100001100101101100100010001001000010010010001100101110101000100001111100101010000101001000010110000100010101100111000101100101010111001011011010110011101011010100100010101111111100010101000001010110101110100101011101011010010000010111101000011100010011101001000100000010010110000101110010010111110000010111111110000000001001000100001010001111001010101011101000000000100111000000100000010110100011101001001000011011100100111001100010000101000011100000111101100110111110100001010010111011001011111101100110101101111000000001100110110101110110010111001000101011110100111011000011111011110100010100101000001000001101101110100101010001001001101110100011101110001000001111001110110001001000010110111110111111111011111001011010111110111011001011111111011110101111011011110001010110010110010001010011111000101101001011100001100101111101000101111100010101101001000011011110110111111010000011000011101010110011011001001110111101101101000010111101101101011001011001001100010101111010011111000011011011100000011000101000001101111001110010101110010000110011111110010001001101011001101101101000010101001000011100100010110101100111110100010001000110000100001011100000000110010011000001011000101001001110111001000111011001011101001110011111011010100011000111000000100101000110101001101110010011110111101111010110010010000011110000000010100000111000101001010110011100110001110111110010000010101011001011011000010110000100011111010010010111101000000111010000110010100011110000100010100110101001010000001110101111111001000110000000101101010000010010011001110000101110111101101001111011010100111011001010101111111100010100001000000011101110110001011001101100100001100110011001101001011010000100110000000111111001000001001001110001000011010101101101011100000010000000101111101110010110001110001110110111001010100010000110011100111010011100111110010011111100100010101101010101000100010001000011101010001110000111011000000010111010111101101010011010110101001111101111010111101110000011100000111101000111110111111110100110000100011101110011101101110000101101111111101100101101010010110001110101011010101100111011011011101010001001100000110011100000011011000010011000000100000101110011101010010001110111011010010101100000110101011011110010000110111011010011111111010001001000110100000001111000011010111011100010000110011010010000100011111110011111001001010000111111101011000000011100010000110110101000110010010001100110011100110101001110001110111110011110011011011110001111101110101110000001001101010110001000001010010000110111111110010000010010000100011110010001011001101010100111011001001000011011100111101000101011111011001000110101010001111001011111010011110010111000011101111010111110101010110101011101100100100010001101111111001000011011101111010101001011010111111101000011101010001100101011101000000010101101000010111011001110000010111101110100010011111010101101101111011010011011100000010101101001000000111011110101011101101100010110001100110001001101001100001100100010000000001011000110111110101011101110100001111010001101111110011000110111000001000110111010000011010100101111011011010001110110111010010010011110100111100011110011110101001110010101010101110010111000100111010111000110011100111001110010101000100000010000111000000110000001100100110101111111111110100101110110000011100000000101001001010011011000000101011010011100010000111011011000110111100100101110110110100001000110011010111010010100110000001100000111110000011001110110101100100111010001111100001000011011101111110010000111010010001111111000111011011111000110011111010010000001011100110010100100000101011100010011011001011110011000110010000000110111000011101101001110011001000000001011111000001011101110001010000010101011111011101100110111000001000001110000110011001101110110110100010001001001111110101001011111110000011000011111101000000010101011011000001010010001011110110100010001110100001110101001100001110111000110001010010011000101111100000111000110000100100001000010000110110000001110011011101011110111001011111001010011111010101100011010010110100110101101010011001111110010100001101000101011110001000001101100110111000010110100010111"
src = [('00', 1498), ('01', 259), ('10', 17), ('11', 380)]
txt_decoded = arithmetic_decode_bin(txt,14,src,9080);
print(txt_decoded)

In [30]:
txt = "011111100000111110100111110000011001001100000110110010101011000000111100000011011110011011111111101010001110111100101001101110101001111101001111100110000001000010101000010101001111110010111000001100010011111011000001011011001010001100111110011100100010010001100000100001011111111001011101000101110011100111000000100100011111011011010011100011111000100110000010110110010100110101001110110110110010111010101000111110100111110000110111010010001011001100010010001001000111110001101101111001011001110100101010010000010110110011110001110011010000100100011000001111101001111100001101110100100010110011011001010100110100010111001001010010101010001001000111110110110100111000111110001110111101001000111101001011100000111000100111010001100010010111100011100100111001100110011100000100110000101101000101101100101110100101001000100100101101100100100011111101010000100101000101101100101001111010111110001001111101001010010011111010100001001000110000010111001111010001101111111001110001110000101110000010100111110001010000101010011010101000010010001100000011001011100111101101011100000011001111110100010111001001011011001010011000100110011101000111101011110110110001010010111111110010111010010011001001111110011111011111111101010001000100111110100111001100100010010001100000101101011100111001000100111100100010110111010100011111100000111110100111110000110000100110100101000100100011000001011100111101010011101011010100111110100111110100111110011001110010011011101001011110111011000011110110010101001111010010100111010011110001111001011010110100011001010010110101110100010110110010100111101011111000100111110110100011100111110001011010101110100011110101011101001010001001011001011011001010010101111001110111110110101110100010110100110110110010101000100100011000000100001100010011111111111101011000001101110110011111001110001101001011000001111010101110100101001110111101010011110111110110101110000011110011101100010010110011001011110111011110010101001010011111000010100101101001101101100101011010000111100000101101100101001010010011101001010011010100110011110100001001110001011010111010001011011001010011100001001010011001111001100000100111100000101101100100101100110111100111110010001011110101111100001001000110000010110110010010110011000010011111000010010110010110110010100111011110101001111011010111010001011011111101001010011101011010001111111101001100001011010001110100100110000101000110011110100011001011110111011110010101001010111011010101011101110110"
src = [(' ', 72190), ('a', 23858), ('b', 5017), ('c', 6697), ('d', 15027), ('e', 36243), ('f', 6113), ('g', 6699), ('h', 19838), ('i', 19165), ('j', 662), ('k', 3070), ('l', 12294), ('m', 7309), ('n', 20475), ('o', 23601), ('p', 4825), ('q', 180), ('r', 15584), ('s', 18060), ('t', 29363), ('u', 9107), ('v', 2433), ('w', 8111), ('x', 412), ('y', 6809), ('z', 158)];
res = huffman_code(txt, src, 1)
huf = [(key, value) for key, value in zip(res.keys(), canonical_code(res.values(), 2, ['0', '1']))]
corr = dict(huf)
decode(txt,corr)

'if you had been in hucks place youd a sneaked down the hill and never told anybody on the robbers you cant do any but mean things and you cant bear to see anybody praised for doing good ones there no thanks as the widow says and tom cuffed sids ears and helped him to the door with several kicks now go and tell auntie if you dare and tomorrow youll catch it some minutes later the widows guests were at the supper table and a dozen children were propped up at little side tables in the same room after the fashion of that country and that day at the proper time mr jones made his little speech'

In [148]:
src = [('a', 407), ('b', 5), ('c', 3150), ('d', 2), ('e', 8), ('f', 3844), ('g', 106251), ('h', 11724), ('i', 3), ('j', 81217), ('k', 2), ('l', 829), ('m', 77), ('n', 3576), ('o', 1242), ('p', 7331), ('q', 533), ('r', 112), ('s', 1), ('t', 4), ('u', 1), ('v', 757), ('w', 8262), ('x', 4063), ('y', 1172), ('z', 56)];
pp = product_prob(src)
res = huffman_code("quijote", pp, 2)
huf = [(key, value) for key, value in zip(res.keys(), canonical_code(res.values(), 2, ['0', '1']))]
cod = [y for x,y in huf];
pp2 = dict(pp);
mean_length(pp, cod) - entropy2(pp2)

0.029262373111501816

In [ ]:
entropy(quijote,2,pre="co")